In [1]:
import numpy as np
import os

import matplotlib.pyplot as plt

# pickle
import pickle

# tf and keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

2023-08-31 20:33:17.943107: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 20:33:18.080375: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 20:33:18.081242: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 20:33:19.008439: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Reading the data

In [ ]:
file = open("Dataset/1661-0.txt", "r", encoding = "utf8")
lines = []

for i in file:
    lines.append(i)
    
data = ""

for i in lines:
    data = ' '. join(lines)
    
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“', '').replace("”", '')

data = data.split()
data = ' '.join(data)
data[:500]

In [ ]:
len(data)

# Preprocessing

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]

In [ ]:
len(sequence_data)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

In [ ]:
sequences = []
for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
    
print("The length of sequences :", len(sequences))
sequences = np.array(sequences)
sequences[:10]

# Train and Test

In [ ]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
    
X = np.array(X)
y = np.array(y)

In [ ]:
print("Data :\n", X[:10])
print()
print("Response :\n", y[:10])

In [ ]:
y = to_categorical(y, num_classes=vocab_size)
y[:10]

# LSTM

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, show_layer_names=True)

# Modeling

In [ ]:
checkpoint = ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
lstm = model.fit(X, y, validation_split=0.2, epochs=150, batch_size=128, shuffle=True, callbacks=[checkpoint]).history

In [ ]:
plt.plot(lstm['loss'])
plt.plot(lstm['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

# Results

In [ ]:
model = load_model("nextword1.h5")
tokenizer = pickle.load(open("tokenizer1.pkl", 'rb'))

In [ ]:
def predict_next_words(model, tokenizer, text) :
    
    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    
    preds = np.argmax(model.predict(sequence))
    predicted_word = ""
    
    for key, value in tokenizer.word_index.items() :
        if value == preds :
            predicted_word = key
            break
    
    return predicted_word

In [ ]:
l = ["The Adventures of", "by Arthur Conan", "it is all", "0"]

for i in l :
    
    text = i
    
    if text == "0" :
        print()
        print("Execution completed...")
        break
    
    else :
        try :
            text = text.split(" ")
            text = text[-3:]
            print("Input :", i)
            print("Output : ", i, predict_next_words(model, tokenizer, text))
            print()
            
        except Exception as e :
            print("Error occured : ", e)
            continue